In [28]:
# import package
# installed via pip
from emtracks.particle import trajectory_solver # main solver object
from emtracks.conversions import one_gev_c2_to_kg # conversion for q factor (transverse momentum estimate)
from emtracks.tools import *#InitConds # initial conditions namedtuple
from emtracks.mapinterp import get_df_interp_func  # factory function for creating Mu2e DS interpolation function
from emtracks.Bdist import get_B_df_distorted
import matplotlib.animation as animation
import numpy as np
from scipy.constants import c, elementary_charge
import pandas as pd
import pickle as pkl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [24,16] # bigger figures
from matplotlib import style
style.use('fivethirtyeight')
import os

from joblib import Parallel, delayed
import multiprocessing
from tqdm.notebook import tqdm


testdir = "/home/darren/Desktop/plots/"
datadir = "/home/shared_data/"
plotdir = datadir+"plots/randomphi/"
mapdir = datadir+"Bmaps/"
date = "/6-20/"
newdir = datadir+'test4/'

In [2]:
def get_uniform_phi(N):
    phis = np.linspace(0, 2*math.pi, N)
    return phis

def get_uniform_theta(N):
    thetas = np.linspace(0, math.pi, N)
    return thetas

In [3]:
def get_b_dis(start_point, end_point, initial_B_list, final_B):
    dflist = []
    objlist = []
    for i in initial_B_list:
        df_Mu2e = pd.read_pickle(mapdir+"Mu2e_DSMap_V13.p")
        B_Mu2e_func = get_df_interp_func(mapdir+"Mu2e_DSMap_V13.p", gauss=False)

        df_Mu2e_dis = get_B_df_distorted(df_Mu2e, v="0", Bz0 = i, Bzf = 0, z0 = start_point, zf = end_point)
        B_Mu2e_dis = get_df_interp_func(df=df_Mu2e_dis, gauss=False)
        
        dflist.append([df_Mu2e, df_Mu2e_dis, i])
        objlist.append([B_Mu2e_func, B_Mu2e_dis, i])
    return (objlist)

In [29]:
def run_solver(phi, theta, field, fielddescrip, N_calc, xnaught, ynaught, znaught, name):
    ic_Mu2e = InitConds(t0=0., tf=4e-8, N_t=N_calc,
                    x0=xnaught, y0=ynaught, z0=znaught,
                    p0=104.96, theta0=theta, phi0=phi)
    e_solver = trajectory_solver(ic_Mu2e, B_func=field, bounds=bounds_Mu2e)
    sol = e_solver.solve_trajectory(verbose = False, atol=1e-10, rtol=1e-10)
    e_solver.dataframe['r'] = ((e_solver.dataframe['x'])**2 + (e_solver.dataframe['y'])**2)**(1/2)
    e_solver.dataframe['rfromcenter'] = ((e_solver.dataframe['x'])**2 + (e_solver.dataframe['y'])**2)**(1/2)
    filename = f'{fielddescrip}_{ic_Mu2e.theta0:0.3f}_{ic_Mu2e.phi0:0.3f}_{ic_Mu2e.x0:0.3f}_{name}.pkl'
    e_solver.to_pickle(newdir+filename)    

In [30]:
def runbothsolvers(phi, theta, field, N_calc, xnaught, ynaught, znaught):
    run_solver(phi, theta, field[0], field[2], N_calc, xnaught, ynaught, znaught, 'nom') #second argument is how many steps in numerical integration
    run_solver(phi, theta, field[1], field[2], N_calc, xnaught, ynaught, znaught, 'dis') #second argument is how many steps in numerical integration

In [33]:
def savepickle(N, fieldvals, ncalc = 20001):
    phis = get_uniform_phi(N)
    thetas = get_uniform_theta(N)
    fields = get_b_dis(3, 14, fieldvals, 0)
    numcpu = multiprocessing.cpu_count()
    
    Parallel(n_jobs = numcpu)(delayed(runbothsolvers)(phi, theta, field, ncalc, 0.054094482, 0.03873037, 5.988900879) for theta in tqdm(thetas, desc = 'theta') for field in fields for phi in phis)


In [34]:
savepickle(50, [3.0, 5.0, 50.0], 10001)

In [11]:
cpu_count = multiprocessing.cpu_count()

64

In [ ]:
def myfunction(index):
    return index

